## Блокнот для прикидки влияния отдельных параметров


In [7]:
# from utils import X,y,cat_cols,fillna,normalize,skew,encode
from task import *
import gc
gc.collect()

0

### Для тестирования используется тренировочный набор
Набор разделён на обучающий и тестовый  в соотношении 3:1 без перемешивания 

In [2]:
def data():
    split = 300_000
    maxlines = 500_000
    d2 = pd.read_csv('datasets/train.csv',skiprows= range(1,split) )
    d1 = pd.read_csv('datasets/train.csv',skiprows= range(split,maxlines) )
    return(d1,d2)  


### Класс для прогона тестов
На экран выводится метрика MAPE на тетсовом наборе 
Кроме того, базовый класс записывает метрику в лог, откуда её можно извлечь

In [3]:
class Validator(Task):

    def __init__( self, data, params = {}):
        super().__init__(data, params)
        self.ds = ScoreSet( data[0],data[1] ,params=self.params)

    def process(self,ds = None ):
        if ds==None:
            ds = self.ds
        ds.set_encoder( self.new_encoder() ).set_model(self.new_model() )
        ds.prepare().fit().predict()
        self.log +=(f"{ds.log}\t MAPE:{ds.scores()}\n")          

    

In [4]:
# params = { 
#             'rounding': { 'odometer':3000,'condition':1},
# #            'skew': {'threshold':4000,'mult':4 },
#             'model': {  'criterion':'poisson','n_estimators':500, 'max_depth':200, 'min_samples_leaf':2 }  
#         }
# v = Validator( data() ,params=params)
# v.process()
# print(v.log)
# del v
# gc.collect()

### Подбор параметров в цикле

In [5]:
params = { 
            'rounding': { 'odometer':3000,'condition':1},
            'model': {  'criterion':'poisson','n_estimators':200, 'max_depth':50, 'min_samples_leaf':2 }  
        }
for msl in [20,40,10]:
    for smooth  in [10,5,20]:
        for ps in [10]:
            params.update({'rounding': { 'odometer':1000 ,'condition':1,'prefix_size':ps}, 
 #                          'skew': {'threshold':th,'mult':mult },
                           'encoder': {'min_samples_leaf':msl,'smoothing':smooth} ,
                           'sellers':{'stages':5}
                          })
            v = Validator( data() ,params=params)
            v.process()
            print(v.log)       
# -- interrupted on step 11 as no further improvements expected             
            

X =  ['year', 'make', 'model', 'abbr_trim', 'body', 'transmission', 'state', 'condition', 'odometer', 'abbr_seller']
	 MAPE:0.1709027649433777

X =  ['year', 'make', 'model', 'abbr_trim', 'body', 'transmission', 'state', 'condition', 'odometer', 'abbr_seller', 'q25', 'q75']
	 MAPE:0.170777741132744

X =  ['year', 'make', 'model', 'abbr_trim', 'body', 'transmission', 'state', 'condition', 'odometer', 'abbr_seller', 'q25', 'q75', 'q25', 'q75']
	 MAPE:0.17114831731317579

X =  ['year', 'make', 'model', 'abbr_trim', 'body', 'transmission', 'state', 'condition', 'odometer', 'abbr_seller', 'q25', 'q75', 'q25', 'q75', 'q25', 'q75']
	 MAPE:0.17273723951367886

X =  ['year', 'make', 'model', 'abbr_trim', 'body', 'transmission', 'state', 'condition', 'odometer', 'abbr_seller', 'q25', 'q75', 'q25', 'q75', 'q25', 'q75', 'q25', 'q75']
	 MAPE:0.17245131847161185

X =  ['year', 'make', 'model', 'abbr_trim', 'body', 'transmission', 'state', 'condition', 'odometer', 'abbr_seller', 'q25', 'q75', 'q25', 

KeyboardInterrupt: 

In [ ]:
# Unfortunately data size proved to be too big for my PC, so I have to gree memory as often as possibly :(
# import gc
# gc.collect()

In [ ]:
v.ds.tr[X]

In [ ]:
# def classify_sellers(ds,stages= 6):
#     abbr_count = ds['abbr_seller'].value_counts()

#     def abbr_power(s):
#         for i in range(stages):
#             if 2**i > abbr_count[s]:
#                 return('ABBR_T_'+str(i) )
#         return( str(s) )

#     ds['abbr_seller']= ds['abbr_seller'].apply(abbr_power)


#     g_abbr = ds.groupby('abbr_power').condition
#     q_abbr = pd.DataFrame( { 'q25': g_abbr.quantile(.25),'q75':g_abbr.quantile(.75)} )
#     #q_abbr.loc['159191']
#     ds[['q25','q75']] = ds.abbr_power.apply(lambda s: q_abbr.loc[s] )

# classify_sellers(v.ds.tr)

In [ ]:
import matplotlib.pyplot as  plt
global X

fis = v.ds.model.feature_importances_.argsort()[3:]
plt.barh(pd.Series(X)[fis], v.ds.model.feature_importances_[fis]);
plt.xlabel("Влияние признаков");

In [ ]:
v.ds.te.model.value_counts()